<a href="https://colab.research.google.com/github/Don864/IntelligentSystemML/blob/main/ISY503_Penanueva_D_Ass2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISY503 Intelligent Systems Assessment 2
# Prepared by Donny Penanueva

# ReadMe
- The car data set we will be using in this lab is provided as a comma-separated file without a header row.
 - The data consists of 205 rows
 - 15 numeric_feature_names
 - 10 categorical_feature_names
-  For each column to have a meaningful header name is provided.
- Header 'price' is the target feature (y-axis)
- This is a Google Colab file (.ipynb) which you can run by going to this website https://colab.google. From the colab website, you can also download this file as .py to run locally in your desired IDE, all dependencies need to install to make the code work properly.

# Task 0
  - This task prepares the data and validate the categories such as numeric and categorical features.
  - The dataset contains 26 columns and the column 'price' (y-axis) is the target. While the 25 columns composed of 15 numeric features and 10 categorical features.
  - Filling missing values with 0 can impact and may produce bias in the model.

  - Numeric Feature Names:
['symboling', 'normalized-losses', 'wheel-base', 'length', 'width', 'height', 'weight', 'engine-size', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke', 'compression-ratio']

- Categorical Feature Names:
['fuel-system', 'make', 'num-doors', 'engine-type', 'fuel-type', 'drive-wheels', 'body-style', 'num-cylinders', 'engine-location', 'aspiration']

# Task 1
Making the best model for numeric features without normalization allowed by changing some of hyperparamaters:

 - Learning rate: Changing the learning rate from 0.01 to 0.001. Setting the learning rate lower can avoid divergence and help stabilize the training.

 - Choice of optimizer: The optimizer used is Adam Optimizer for its adaptive learning rates, and improved performance. Tried to use Adagrad optimizer but did not get the best result, so Adam is the preferred  option for this model and dataset. When using the Gradient Descent Optimizer (SGD), the result has an Nan loss error , which makes it not suitable for this model of dataset.                

- Hidden layer dimensions: Increased the hidden layer dimensions to [128, 64] to learn complex patterns in the data.

- Batch size: For a small dataset 16 batch size is reasonable which can also help in avoiding overfitting.

- Num training steps: The training steps used is 20000 which gives a better result. Initially tried to use 12000 and 15000 but it was giving a message to increase the steps.


 Hyperparamaters     value
---------------     --------
 learning rate      .001
 batch size         16
 hidden layer       [128, 64]  
 num training steps 20000  


# Task 2
 With the use of normalization the results are better and improved compared to task 1. By normalizing the model makes it more efficient and effective. Adam optimizer mostly works well with normalization features. Meanwhile Gradient Descent Optimizer (SGD) is not suitable for this model which is giving a Nan loss during training.


 # Task 3
 The categorical features is used in this task.

 # Task 4
 In this task all the features are consolidated to achieve the best results for this model. In this code "numeric_only=True" is added to avoid error when running the code below:
normalizer_fn=lambda val: (val - x_df.mean(numeric_only=True)[feature_name]) / (epsilon + x_df.std(numeric_only=True)[feature_name]))



In [ ]:

### Task 0
print('## Task 0 Preparing the data')
# Importing the dependencies
import numpy as np
import pandas as pd
import math

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Providing the names for the columns
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']


# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                        sep=',', names=feature_names, header=None, encoding='latin-1')


LABEL = 'price'

# Randomize the data.
car_data = car_data.reindex(np.random.permutation(car_data.index))

# Printing the number of examples = 205
print("Data set loaded. Num examples: ", len(car_data))

print(car_data.head())

# Use type() to confirm that each column is of the expected type (numeric or object)
for column in car_data.columns:
    print(f"Column '{column}' is of type: {type(car_data[column].iloc[0])}")

# Manually curate a list of numeric feature names based on the column types
numeric_feature_names = ['symboling','normalized-losses','wheel-base', 'length', 'width', 'height', 'weight',
                         'engine-size','horsepower','peak-rpm',
                         'city-mpg','highway-mpg', 'bore','stroke','compression-ratio']
categorical_feature_names = list(set(feature_names) - set(numeric_feature_names) - set([LABEL]))


#To show the length of numeric_feature_names and categorical_feature_names
print("\nNumeric Feature Names Length:")
print(len(numeric_feature_names))
print("\nCategorical Feature Names Length:")
print(len(categorical_feature_names))


# Confirm the lists
print("\nNumeric Feature Names:")
print(numeric_feature_names)
print("\nCategorical Feature Names:")
print(categorical_feature_names)

# Testing if the correct solution will pass these assert statements.
assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

# Run to inspect numeric features.
car_data[numeric_feature_names]

# Run to inspect categorical features.
car_data[categorical_feature_names]


# # Coerce the numeric features to numbers. This is necessary because the model
# # crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
  car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors='coerce')

# # Fill missing values with 0.
car_data.fillna(0, inplace=True)




# Task 1 Numeric feaures without normalization.
# By using AdamOptimizer instead of GradientDescentOptimizer fixed the Nan loss error
print('## Task 1 Numeric features without normalization')

batch_size = 16

print(numeric_feature_names)
x_df = car_data[numeric_feature_names]
y_series = car_data['price']

# Create input_fn's so that the estimator knows how to read in your data.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    batch_size=batch_size,
    shuffle=False)

# Feature columns allow the model to parse the data, perform common
# preprocessing, and automatically generate an input layer for the tf.Estimator.
model_feature_columns = [
    tf.feature_column.numeric_column(feature_name) for feature_name in numeric_feature_names
]
print('model_feature_columns', model_feature_columns)

# Use AdamOptimizer instead of GradientDescentOptimizer
est = tf.estimator.DNNRegressor(
    feature_columns=model_feature_columns,
    hidden_units=[128, 64],   # Increase the size of the hidden layers
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), # Adjust the learning rate
  )

# TRAIN
num_print_statements = 10
num_training_steps = 20000
for _ in range(num_print_statements):
  est.train(train_input_fn, steps=num_training_steps // num_print_statements)
  scores = est.evaluate(eval_input_fn)

  # The `scores` dictionary has several metrics automatically generated by the
  # canned Estimator.
  # `average_loss` is the average loss for an individual example.
  # `loss` is the summed loss for the batch.
  # In addition to these scalar losses, you may find the visualization functions
  # in the next cell helpful for debugging model quality.
  print('scores', scores)

# Visualize the data

from matplotlib import pyplot as plt


def scatter_plot_inference_grid(est, x_df, feature_names):
  """Plots the predictions of the model against each feature.

  Args:
    est: The trained tf.Estimator.
    x_df: The pandas dataframe with the input data (used to create
      predict_input_fn).
    feature_names: An iterable of string feature names to plot.
  """
  def scatter_plot_inference(axis,
                             x_axis_feature_name,
                             y_axis_feature_name,
                             predictions):
    """Generate one subplot."""
    # Plot the real data in grey.
    y_axis_feature_name = 'price'
    axis.set_ylabel(y_axis_feature_name)
    axis.set_xlabel(x_axis_feature_name)
    axis.scatter(car_data[x_axis_feature_name],
                 car_data[y_axis_feature_name],
                 c='grey')

    # Plot the predicted data in orange.
    axis.scatter(car_data[x_axis_feature_name], predictions, c='orange')

  predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    batch_size=batch_size,
    shuffle=False)

  predictions = [
    x['predictions'][0]
    for x in est.predict(predict_input_fn)
  ]

  num_cols = 3
  num_rows = int(math.ceil(len(feature_names)/float(num_cols)))
  f, axarr = plt.subplots(num_rows, num_cols)
  size = 4.5
  f.set_size_inches(num_cols*size, num_rows*size)

  for i, feature_name in enumerate(numeric_feature_names):
    axis = axarr[int(i/num_cols), i%num_cols]
    scatter_plot_inference(axis, feature_name, 'price', predictions)
  plt.show()

scatter_plot_inference_grid(est, x_df, numeric_feature_names)


# Task 2  with Normalization
print('## Task 2 with Normalization')

# This 1D visualization of each numeric feature might inform your normalization
# decisions.
for feature_name in numeric_feature_names:
  car_data.hist(column=feature_name)


# This does Z-score normalization since the distributions for most features looked
# roughly normally distributed.

# Z-score normalization subtracts the mean and divides by the standard deviation,
# to give a roughly standard normal distribution (mean = 0, std = 1) under a
# normal distribution assumption. Epsilon prevents divide by zero.



batch_size = 16

print(numeric_feature_names)
x_df = car_data[numeric_feature_names]
y_series = car_data['price']

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    batch_size=batch_size,
    shuffle=False)

# Epsilon prevents divide by zero.
epsilon = 0.000001
model_feature_columns = [
    tf.feature_column.numeric_column(feature_name,
                                     normalizer_fn=lambda val: (val - x_df.mean()[feature_name]) / (epsilon + x_df.std()[feature_name]))
    for feature_name in numeric_feature_names
]
print('model_feature_columns', model_feature_columns)

est = tf.estimator.DNNRegressor(
    feature_columns=model_feature_columns,
    hidden_units=[128, 64],
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
  )

# TRAIN
num_print_statements = 10
num_training_steps = 20000
for _ in range(num_print_statements):
  est.train(train_input_fn, steps=num_training_steps // num_print_statements)
  scores = est.evaluate(eval_input_fn)


  print('scores', scores)

scatter_plot_inference_grid(est, x_df, numeric_feature_names)


#Task 3
# We have the full list of values that each feature takes on, and the list is
# relatively small so we use categorical_column_with_vocabulary_list.

print('## Task 3 ')
batch_size = 16

x_df = car_data[categorical_feature_names]
y_series = car_data['price']

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    batch_size=batch_size,
    shuffle=False)

model_feature_columns = [
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            feature_name, vocabulary_list=car_data[feature_name].unique()))
    for feature_name in categorical_feature_names
]
print('model_feature_columns', model_feature_columns)

est = tf.estimator.DNNRegressor(
    feature_columns=model_feature_columns,
    hidden_units=[128, 64],
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
  )

# TRAIN
num_print_statements = 10
num_training_steps = 20000
for _ in range(num_print_statements):
  est.train(train_input_fn, steps=num_training_steps // num_print_statements)
  scores = est.evaluate(eval_input_fn)


  print('scores', scores)



# Task 4
# This is a first pass at a model that uses all the features.
print('## Task 4 Consolidate all the features')


batch_size = 16

x_df = car_data[numeric_feature_names + categorical_feature_names]
y_series = car_data['price']

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    batch_size=batch_size,
    shuffle=False)

epsilon = 0.000001
model_feature_columns = [
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            feature_name, vocabulary_list=car_data[feature_name].unique()))
    for feature_name in categorical_feature_names
] + [
    tf.feature_column.numeric_column(feature_name,
                                     normalizer_fn=lambda val: (val - x_df.mean(numeric_only=True)[feature_name]) / (epsilon + x_df.std(numeric_only=True)[feature_name]))
    for feature_name in numeric_feature_names
]


print('model_feature_columns', model_feature_columns)

est = tf.estimator.DNNRegressor(
    feature_columns=model_feature_columns,
    hidden_units=[128, 64],
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001),
  )

# TRAIN
num_print_statements = 10
num_training_steps = 20000
for _ in range(num_print_statements):
  est.train(train_input_fn, steps=num_training_steps // num_print_statements)
  scores = est.evaluate(eval_input_fn)


  print('scores', scores)



